# Importing Libraries

In [6]:
pip install --upgrade git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to c:\users\86177\appdata\local\temp\pip-req-build-_1eb3ggv
  Resolved https://github.com/jpmml/sklearn2pmml.git to commit 1c834d43ce7920b6ff900b26fda6e06820606210
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy>=1.13.3 in c:\users\86177\anaconda3\lib\site-packages (from scikit-learn>=0.18.0->sklearn2pmml==0.92.0) (1.20.3)



  Running command git clone -q https://github.com/jpmml/sklearn2pmml.git 'C:\Users\86177\AppData\Local\Temp\pip-req-build-_1eb3ggv'


In [7]:
# pip install sklearn2pmml

In [8]:
#pip install plotly==5.14.1

In [9]:
#pip install emoji

In [10]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import emoji
import string
import nltk
from PIL import Image
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\86177\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words = set(stopwords.words('english'))

# Getting data

In [12]:
data = pd.read_csv('cyberbullying_tweets.csv')

## Renaming columns

In [13]:
data = data.rename(columns={'tweet_text': 'text', 'cyberbullying_type': 'sentiment'})

### Adding Encoded column for sentiments

In [14]:
data["sentiment_encoded"] = data['sentiment'].replace({"religion": 1, "age": 2, "ethnicity": 3, "gender": 4, "other_cyberbullying": 5,"not_cyberbullying": 6})

# Preprocessing of Text

## Function to Remove Emojis

In [15]:
def strip_emoji(text):
    return emoji.replace_emoji(text,replace="")

## Fucntion to Convert text to lowercase, remove (/r, /n  characters), URLs, non-utf characters, Numbers, punctuations,stopwords

In [16]:
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    text = re.sub(r'(.)1+', r'1', text)
    text = re.sub('[0-9]+', '', text)
    stopchars= string.punctuation
    table = str.maketrans('', '', stopchars)
    text = text.translate(table)
    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)
    return text

## Function to remove contractions

In [17]:
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

## Function to Clean Hashtags

In [18]:
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet))
    return new_tweet2

## Function to Filter Special Characters such as $, &

In [19]:
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

## Function to remove mutiple sequence spaces

In [20]:
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)


## Function to apply stemming to words

In [21]:
def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

## Function to apply lemmatization to words

In [22]:
def lemmatize(text):
    tokenized = nltk.word_tokenize(text)
    lm = WordNetLemmatizer()
    return ' '.join([lm.lemmatize(words) for words in tokenized])

## Function to Preprocess the text by applying all above functions (Choose one)

In [23]:
def preprocess(text): #(initial one)
    text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    #text = stemmer(text)
    #text = lemmatize(text)
    return text

In [24]:
def preprocess1(text): #(initial one)
    #text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    #text = stemmer(text)
    #text = lemmatize(text)
    return text

In [25]:
def preprocess(df1):
    
    df1['cleaned_text'] = df1['text'].apply(strip_emoji)
    df1['cleaned_text'] = df1['text'].apply(decontract)
    df1['cleaned_text'] = df1['text'].apply(strip_all_entities)
    df1['cleaned_text'] = df1['text'].apply(clean_hashtags)
    df1['cleaned_text'] = df1['text'].apply(filter_chars)
    df1['cleaned_text'] = df1['text'].apply(remove_mult_spaces)
    df1['cleaned_text'] = df1['text'].apply(stemmer)
    df1['cleaned_text'] = df1['text'].apply(lemmatize)
    return df1['cleaned_text']

In [26]:
def preprocess(df1):

    if type(df1) == type('string'):
            df1 = strip_emoji(df1)
            df1 = decontract(df1)
            df1 = strip_all_entities(df1)
            df1 = clean_hashtags(df1)
            df1 = filter_chars(df1)
            df1 = remove_mult_spaces(df1)
            df1 = stemmer(df1)
            df1 = lemmatize(df1)
            return df1
    else:
            df1['cleaned_text'] = df1['text'].apply(strip_emoji)
            df1['cleaned_text'] = df1['text'].apply(decontract)
            df1['cleaned_text'] = df1['text'].apply(strip_all_entities)
            df1['cleaned_text'] = df1['text'].apply(clean_hashtags)
            df1['cleaned_text'] = df1['text'].apply(filter_chars)
            df1['cleaned_text'] = df1['text'].apply(remove_mult_spaces)
            df1['cleaned_text'] = df1['text'].apply(stemmer)
            df1['cleaned_text'] = df1['text'].apply(lemmatize)
            return df1['cleaned_text']


In [27]:
userinput = "pussy pussy #$"

In [28]:
type("pussy pussy #$") == type('string')

True

In [29]:
data['cleaned_text'] = data['text'].apply(preprocess)
data.head()

,text,sentiment,sentiment_encoded,cleaned_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,6,word katandandr food crapilici mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,6,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,6,classi whore red velvet cupcak
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,6,meh p thank head concern anoth angri dude twitter
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,6,isi account pretend kurdish account like islam...


## Dealing with Duplicates

In [30]:
data.drop_duplicates("cleaned_text", inplace=True)

# Tokenization

In [31]:
data['tweet_list'] = data['cleaned_text'].apply(word_tokenize)
data.head()

,text,sentiment,sentiment_encoded,cleaned_text,tweet_list
0,"In other words #katandandre, your food was cra...",not_cyberbullying,6,word katandandr food crapilici mkr,"[word, katandandr, food, crapilici, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,6,aussietv white mkr theblock imacelebrityau tod...,"[aussietv, white, mkr, theblock, imacelebritya..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,6,classi whore red velvet cupcak,"[classi, whore, red, velvet, cupcak]"
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,6,meh p thank head concern anoth angri dude twitter,"[meh, p, thank, head, concern, anoth, angri, d..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,6,isi account pretend kurdish account like islam...,"[isi, account, pretend, kurdish, account, like..."


# Checking length of various tweet texts

In [32]:
text_len = []
for text in data.tweet_list:
    tweet_len = len(text)
    text_len.append(tweet_len)
data['text_len'] = text_len

# Removing text without words

In [33]:
data = data[data['text_len']!=0]

In [34]:
data.shape

(44650, 6)

In [35]:
data.head()

,text,sentiment,sentiment_encoded,cleaned_text,tweet_list,text_len
0,"In other words #katandandre, your food was cra...",not_cyberbullying,6,word katandandr food crapilici mkr,"[word, katandandr, food, crapilici, mkr]",5
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,6,aussietv white mkr theblock imacelebrityau tod...,"[aussietv, white, mkr, theblock, imacelebritya...",11
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,6,classi whore red velvet cupcak,"[classi, whore, red, velvet, cupcak]",5
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,6,meh p thank head concern anoth angri dude twitter,"[meh, p, thank, head, concern, anoth, angri, d...",9
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,6,isi account pretend kurdish account like islam...,"[isi, account, pretend, kurdish, account, like...",8


In [36]:
sentiments = ["religion", "age", "ethnicity", "gender", "other_cyberbullying","not_cyberbullying"]

# Splitting Data into Train and Test Sets

In [37]:
X,Y = data['cleaned_text'],data['sentiment_encoded']

In [38]:
Y

0        6
1        6
2        6
3        6
4        6
        ..
47687    3
47688    3
47689    3
47690    3
47691    3
Name: sentiment_encoded, Length: 44650, dtype: int64

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify =Y, random_state = 42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(31255,) (31255,) (13395,) (13395,)


In [40]:
y_train

780      6
17323    1
39189    2
19066    1
44698    3
        ..
42704    3
32187    2
14041    4
33441    2
39289    2
Name: sentiment_encoded, Length: 31255, dtype: int64

## tf-idf Vectorization

In [41]:
from sklearn2pmml import PMMLPipeline, sklearn2pmml

In [42]:
XX = data[['cleaned_text']]

XX.columns

Index(['cleaned_text'], dtype='object')

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn2pmml.feature_extraction.text import Splitter
#vectorizer = TfidfVectorizer(analyzer = "word", token_pattern = None, tokenizer = Splitter())

In [44]:
# pip install --upgrade nyoka

In [45]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import LinearSVC
from nyoka import skl_to_pmml

In [46]:
X_train

12740    taking brown sticky joke gay people coprophili...
32959    bullied class asked teacher could call mom emo...
38128    definitely believe agressive strict laws punis...
962            see daesh fleeing kobane mosule countryside
10861    wo answer like feminists answer exposes opposi...
                               ...                        
40757    good luck leads silver except fuck calski dumb...
23370    otherwise atheists support radical christians ...
11262                  jus gonna say itagaindumb bitch mkr
13808    niggaz call females bitches time homeboys let ...
47458    kenyans turned bants racism color discriminati...
Name: cleaned_text, Length: 31299, dtype: object

In [51]:
X_train11 = X_train.apply(preprocess)
X_train11

TypeError: string indices must be integers

In [51]:
vectorizer = TfidfVectorizer(preprocessor=preprocess1)

In [51]:
# mod = LinearSVC(C=1, loss='hinge')
# here you can use the key classifier, if suitable
pipeline = Pipeline([("vect", vectorizer),("model", DecisionTreeClassifier()) ])
#training the model



In [52]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocess1 at 0x00000236772F2160>)),
                ('model', DecisionTreeClassifier())])

In [53]:
# exporting the model
skl_to_pmml(pipeline,XX.columns,"sentiment_encoded","lsvc_tfidf.pmml")

: 

: 

In [80]:
mod = LinearSVC(C=1, loss='hinge')

In [82]:
prep = preprocess

In [119]:
from sklearn.preprocessing import FunctionTransformer

In [104]:

# here you can use the key classifier, if suitable
pipeline = Pipeline([('prep',
FunctionTransformer(preprocess)),("vect", vectorizer),("model", mod) ])
#training the model
pipeline.fit(data,Y)
# exporting the model
#skl_to_pmml(pipeline,XX.columns,"sentiment_encoded","lsvc_tfidf.pmml")


Pipeline(steps=[('prep',
                 FunctionTransformer(func=<function preprocess at 0x00000217A7C364C0>)),
                ('vect',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<sklearn2pmml.feature_extraction.text.Splitter object at 0x00000217A484A160>)),
                ('model', LinearSVC(C=1, loss='hinge'))])

In [164]:
# here you can use the key classifier, if suitable
pipeline = Pipeline([("vect", vectorizer),("model", LogisticRegression()) ])
#training the model
#pipeline.fit(data,Y)
# exporting the model
#skl_to_pmml(pipeline,XX.columns,"sentiment_encoded","lsvc_tfidf.pmml")

TypeError: 'TfidfVectorizer' object is not callable

In [48]:

from sklearn2pmml import make_pmml_pipeline, sklearn2pmml


In [ ]:
pip install -q git+https://github.com/gmihaila/ml_things.git

In [ ]:
pip install --upgrade scikit-learn

In [129]:
XX.columns.values

array(['cleaned_text'], dtype=object)

In [132]:
Y.name

'sentiment_encoded'

In [52]:
pmml_pipe = make_pmml_pipeline( vectorizer, DecisionTreeClassifier())

In [53]:
sklearn2pmml(pmml_pipe, 'pmml_model11.pmml', with_repr=True)

Standard output is empty
Standard error:
Exception in thread "main" java.lang.IllegalArgumentException: Array or list attribute 'sklearn2pmml.pipeline.PMMLPipeline.active_fields' contains an unsupported value (Python class sklearn.tree._classes.DecisionTreeClassifier)
	at org.jpmml.python.CastFunction.apply(CastFunction.java:47)
	at com.google.common.collect.Lists$TransformingRandomAccessList$1.transform(Lists.java:651)
	at com.google.common.collect.TransformedIterator.next(TransformedIterator.java:47)
	at sklearn2pmml.pipeline.PMMLPipeline.initFeatures(PMMLPipeline.java:680)
	at sklearn2pmml.pipeline.PMMLPipeline.encodePMML(PMMLPipeline.java:151)
	at com.sklearn2pmml.Main.run(Main.java:91)
	at com.sklearn2pmml.Main.main(Main.java:66)
Caused by: java.lang.ClassCastException: Cannot cast sklearn.tree.TreeClassifier to java.lang.String
	at java.lang.Class.cast(Unknown Source)
	at org.jpmml.python.CastFunction.apply(CastFunction.java:45)
	... 6 more



RuntimeError: The SkLearn2PMML application has failed. The Java executable should have printed more information about the failure into its standard output and/or standard error streams

In [108]:
param_grid = {'clf': [LogisticRegression(penalty='l2', max_iter=500)]
    , 'clf__C': [0.2, 0.4, 0.6, 0.8]

    }

In [ ]:
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring='f1_macro'
                    , cv=6, verbose=5, return_train_score=True, n_jobs=2)

In [136]:
grid

GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('prep',
                                        FunctionTransformer(func=<function preprocess at 0x00000217A7C364C0>)),
                                       ('vect',
                                        TfidfVectorizer(token_pattern=None,
                                                        tokenizer=<sklearn2pmml.feature_extraction.text.Splitter object at 0x00000217A484A160>)),
                                       ('model', LogisticRegression())]),
             n_jobs=2,
             param_grid={'clf': [LogisticRegression(max_iter=500)],
                         'clf__C': [0.2, 0.4, 0.6, 0.8]},
             return_train_score=True, scoring='f1_macro', verbose=5)

In [165]:
sklearn2pmml(pipeline, "pre_tfidf_log.pmml")

TypeError: The pipeline object is not an instance of PMMLPipeline. Use the 'sklearn2pmml.make_pmml_pipeline(obj)' utility function to translate a regular Scikit-Learn pipeline or estimator to a PMML pipeline

In [178]:
sklearn2pmml(pipeline, "pre_tfidf_lsvc.pmml")

TypeError: The pipeline object is not an instance of PMMLPipeline. Use the 'sklearn2pmml.make_pmml_pipeline(obj)' utility function to translate a regular Scikit-Learn pipeline or estimator to a PMML pipeline

In [139]:
from sklearn2pmml.preprocessing import ReplaceTransformer, StringNormalizer

In [142]:


mapper = DataFrameMapper([
  (["Input Text"], [StringNormalizer("lowercase", trim_blanks = True), ReplaceTransformer(' +', ' '), ReplaceTransformer('[^A-Za-z]+', ' ')])
])
pipeline1122 = PMMLPipeline([
  ("mapper", mapper),
  ('vect', CountVectorizer(ngram_range=(1,2))), 
  ('tfidf',vectorizer), 
  ('clf', LogisticRegression(C=1))
])
pipeline1122.fit(X, Y)

KeyError: "None of [Index(['Input Text'], dtype='object')] are in the [index]"

In [177]:
skl_to_pmml(pipeline,XX.columns,"sentiment_encoded","pre_tfidf_lsvc.pmml")

AttributeError: 'LogisticRegression' object has no attribute 'classes_'

In [107]:
pipeline.predict(data1)

array([4], dtype=int64)

In [128]:

# tf_idf = TfidfVectorizer()
# pipeline = PMMLPipeline([ ("vect", tf_idf) ])
# #training the model
# pipeline.fit(X_train)
# # exporting the model
# sklearn2pmml(pipeline, 'tfidf.pmml', with_repr = True)

In [148]:
tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(X_train)
X_test_tf = tf_idf.transform(X_test)
print(X_train_tf.shape)
print(X_test_tf.shape)

(31255, 29111)
(13395, 29111)


In [129]:
# print(X_train_tf)

## Support Vector Classifier

In [146]:
lin_svc = LinearSVC(C=1, loss='hinge')

# Model use

In [149]:
# Train
lin_svc.fit(X_train_tf,y_train)
y_pred = lin_svc.predict(X_test_tf)

# Input

In [143]:
data1 = pd.read_csv('test.csv')
data1 = data1.rename(columns={'tweet_text': 'text'})
data1['cleaned_text'] = data1['text'].apply(preprocess)
data1.head()

,text,cleaned_text
0,I want hit you,want hit


In [150]:
test11 = data1['cleaned_text']
test11_tf = tf_idf.transform(test11)

In [170]:
vectorizer.fit_transform(X_train)

<31255x29052 sparse matrix of type '<class 'numpy.float64'>'
	with 379731 stored elements in Compressed Sparse Row format>

In [174]:
test1111_idf = vectorizer.transform(test11)

In [172]:
test11

0    want hit
Name: cleaned_text, dtype: object

In [152]:
print(test11_tf)

  (0, 27709)	0.5576538838546059
  (0, 11423)	0.8300735785590779


In [176]:
print(test1111_idf)

  (0, 27661)	0.5576538838546059
  (0, 11403)	0.8300735785590779


In [153]:
y_pred1 = lin_svc.predict(test11_tf)

In [154]:
print(y_pred1)

[5]


In [ ]:
int(np.asarray(y_pred))